In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter, defaultdict

from utils.schemas import *
from utils.functions import *

In [2]:
data_folder = 'input'
schema_train_transaction.update(schema_train_identity)
schema_test_transaction.update(schema_test_identity)

In [3]:
train = pd.read_csv(data_folder+'/train_merged.csv', dtype = schema_train_transaction)
test = pd.read_csv(data_folder+'/test_merged.csv', dtype = schema_test_transaction)

In [4]:
train.shape, test.shape

((590540, 433), (506691, 432))

In [6]:
cat_ft = ['ProductCD', 'P_emaildomain', 'R_emaildomain'] + ['card{}'.format(i) for i in range(1, 7)] \
+ ['addr1', 'addr2'] + ['M{}'.format(i) for i in range(1, 10)] + ['DeviceType', 'DeviceInfo'] \
+ ['id_{}'.format(i) for i in range(12, 39)]

In [7]:
len(cat_ft)

49

In [8]:
len(train.card6.value_counts())
# card1 identificador de algo?

4

In [9]:
re_col_list = ['card', 'C', 'D', 'M', 'V', 'id', 'add', 'dist', 'P', 'R']

for re_col in re_col_list:
    print(train[[x for x in train.columns if re_col in x]].head())

   card1  card2  card3       card4  card5   card6
0  13926    NaN  150.0    discover  142.0  credit
1   2755  404.0  150.0  mastercard  102.0  credit
2   4663  490.0  150.0        visa  166.0   debit
3  18132  567.0  150.0  mastercard  117.0   debit
4   4497  514.0  150.0  mastercard  102.0  credit
  ProductCD   C1   C2   C3   C4   C5   C6   C7   C8   C9  C10  C11  C12   C13  \
0         W  1.0  1.0  0.0  0.0  0.0  1.0  0.0  0.0  1.0  0.0  2.0  0.0   1.0   
1         W  1.0  1.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  1.0  0.0   1.0   
2         W  1.0  1.0  0.0  0.0  0.0  1.0  0.0  0.0  1.0  0.0  1.0  0.0   1.0   
3         W  2.0  5.0  0.0  0.0  0.0  4.0  0.0  0.0  1.0  0.0  1.0  0.0  25.0   
4         H  1.0  1.0  0.0  0.0  0.0  1.0  0.0  1.0  0.0  1.0  1.0  0.0   1.0   

   C14  
0  1.0  
1  1.0  
2  1.0  
3  1.0  
4  1.0  
   TransactionDT ProductCD     D1     D2    D3    D4   D5  D6  D7  D8  D9  \
0          86400         W   14.0    NaN  13.0   NaN  NaN NaN NaN NaN NaN   
1    

In [74]:
list(train.id_33.value_counts().index)

['1920x1080',
 '1366x768',
 '1334x750',
 '2208x1242',
 '1440x900',
 '1600x900',
 '2048x1536',
 '1280x800',
 '2560x1600',
 '2560x1440',
 '2880x1800',
 '1280x1024',
 '1680x1050',
 '1136x640',
 '2436x1125',
 '1280x720',
 '1920x1200',
 '2001x1125',
 '1024x768',
 '2220x1080',
 '5120x2880',
 '3360x2100',
 '2732x2048',
 '1366x767',
 '3840x2160',
 '855x480',
 '2736x1824',
 '1360x768',
 '4096x2304',
 '2224x1668',
 '2220x1081',
 '2160x1440',
 '2961x1442',
 '3200x1800',
 '2560x1080',
 '2562x1442',
 '1600x1200',
 '1280x768',
 '3000x2000',
 '1024x600',
 '1280x1025',
 '1152x864',
 '3440x1440',
 '801x480',
 '2400x1350',
 '960x540',
 '2672x1440',
 '1919x1079',
 '1365x768',
 '2048x1152',
 '1919x1080',
 '2960x1440',
 '1920x1281',
 '1921x1081',
 '1536x864',
 '1280x960',
 '1152x720',
 '3840x2400',
 '2112x1188',
 '2880x1620',
 '1728x972',
 '1502x844',
 '2048x1280',
 '1364x768',
 '1280x1023',
 '1599x900',
 '2049x1152',
 '1920x1079',
 '2400x1600',
 '1281x801',
 '1024x640',
 '3360x1890',
 '1600x899',
 '960x64

In [75]:
train.shape

(590540, 433)

In [10]:
# Posible metodo para hacer time-split crossvalidation
rows = train.shape[0]
for i in range(5):
    print((rows/5)*i, (rows/5)*(i+1))

0.0 118108.0
118108.0 236216.0
236216.0 354324.0
354324.0 472432.0
472432.0 590540.0


In [77]:
for ft in cat_ft:
    v_c = train[ft].value_counts(dropna=False)
    print('LEN de {0}: {1}'.format(ft, len(v_c)))
    print('Suma de NA: {}'.format(train[ft].isna().sum()))
    print(v_c,'\n')

LEN de ProductCD: 5
Suma de NA: 0
W    439670
C     68519
R     37699
H     33024
S     11628
Name: ProductCD, dtype: int64 

LEN de P_emaildomain: 60
Suma de NA: 94456
gmail.com           228355
yahoo.com           100934
NaN                  94456
hotmail.com          45250
anonymous.com        36998
aol.com              28289
comcast.net           7888
icloud.com            6267
outlook.com           5096
msn.com               4092
att.net               4033
live.com              3041
sbcglobal.net         2970
verizon.net           2705
ymail.com             2396
bellsouth.net         1909
yahoo.com.mx          1543
me.com                1522
cox.net               1393
optonline.net         1011
charter.net            816
live.com.mx            749
rocketmail.com         664
mail.com               559
earthlink.net          514
gmail                  496
outlook.es             438
mac.com                436
juno.com               322
aim.com                315
hotmail.es           

In [11]:
first_na_impute_cat = {
    'P_emaildomain': 'na.na',
    'R_emaildomain': 'na.na',
    'card2': -1,
    'card3': -1,
    'card4': 'U',
    'card5': -1,
    'card6': 'U',
    'addr1': -1,
    'addr2': -1,
    'M1': 'U',
    'M2': 'U',
    'M3': 'U',
    'M4': 'M3',
    'M5': 'U',
    'M6': 'U',
    'M7': 'U',
    'M8': 'U',
    'M9': 'U',
    'DeviceType': 'U',
    'DeviceInfo': 'U',
    'id_12': 'U',
    'id_13': -1,
    'id_14': -1,
    'id_15': 'U',
    'id_16': 'U',
    'id_17': -1,
    'id_18': -1,
    'id_19': -1,
    'id_20': -1,
    'id_21': -1,
    'id_22': -1,
    'id_23': 'IP_PROXY:NA',
    'id_24': -1,
    'id_25': -1,
    'id_26': -1,
    'id_27': 'U',
    'id_28': 'U',
    'id_29': 'U',
    'id_30': 'U',
    'id_31': 'U',
    'id_32': 99,
    'id_33': '9999x9999',
    'id_34': 'match_status:-2',
    'id_35': 'U',
    'id_36': 'U',
    'id_37': 'U',
    'id_38': 'U',
}

In [12]:
train[first_na_impute_cat.keys()].head()

,P_emaildomain,R_emaildomain,card2,card3,card4,card5,card6,addr1,addr2,M1,...,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38
0,NaN,NaN,NaN,150.0,discover,142.0,credit,315.0,87.0,T,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,gmail.com,NaN,404.0,150.0,mastercard,102.0,credit,325.0,87.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,outlook.com,NaN,490.0,150.0,visa,166.0,debit,330.0,87.0,T,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,yahoo.com,NaN,567.0,150.0,mastercard,117.0,debit,476.0,87.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,gmail.com,NaN,514.0,150.0,mastercard,102.0,credit,420.0,87.0,NaN,...,NotFound,Android 7.0,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T


In [209]:
train_imputed = train.fillna(first_na_impute_cat)

In [200]:
# n_cut = 64
# y = pd.cut(train_imputed['DeviceInfo'], bins=n_cut, labels=list(range(n_cut-1, -1, -1)), include_lowest=True)
# train_imputed['DeviceInfo_2'] = y

In [274]:
def freq_encoder(df, label, new_label, min_freq = 0.001):
    rows = df.shape[0]
    n = 0
    dict_fe = dict()
    vc = df[label].value_counts()
    for i, j in zip(vc.index, vc):
        ratio = j/rows
        if ratio > min_freq:
            dict_fe[i] = n
            n += 1
        else:
            dict_fe[i] = n
        
    if n < 2**8:
        _d_type = 'uint8'
    elif n >= 2**8 and n < 8**16:
        _d_type = 'uint16'
    elif n >= 2**16 and n < 8**32:
        _d_type = 'uint32'
    else:
        _d_type = 'uint64'
        
    df[new_label] = df[label].apply(lambda x: dict_fe[x]).astype(_d_type)
    
    n = 0
    dict_fe = dict()
    vc = df[label].value_counts()
    for i, j in zip(vc.index, vc):
        ratio = j/rows
        if ratio > min_freq:
            dict_fe[i] = n
            n += 1
        else:
            dict_fe[i] = n
            
    df[new_label] = df[label].apply(lambda x: dict_fe[x]).astype(_d_type)
    
    return df

In [241]:
train_imputed = freq_encoder(train_imputed, 'DeviceInfo', 'DeviceInfo', min_freq = 0.001)
len(train_imputed['DeviceInfo'].value_counts())

8

In [15]:
for ft in cat_ft:
    v_c = train_imputed[ft].value_counts(dropna=False)
    print('LEN de {0}: {1}'.format(ft, len(v_c)))
    print('Suma de NA: {}'.format(train_imputed[ft].isna().sum()))
    print(v_c,'\n')

LEN de ProductCD: 5
Suma de NA: 0
W    439670
C     68519
R     37699
H     33024
S     11628
Name: ProductCD, dtype: int64 

LEN de P_emaildomain: 60
Suma de NA: 0
gmail.com           228355
yahoo.com           100934
na.na                94456
hotmail.com          45250
anonymous.com        36998
aol.com              28289
comcast.net           7888
icloud.com            6267
outlook.com           5096
msn.com               4092
att.net               4033
live.com              3041
sbcglobal.net         2970
verizon.net           2705
ymail.com             2396
bellsouth.net         1909
yahoo.com.mx          1543
me.com                1522
cox.net               1393
optonline.net         1011
charter.net            816
live.com.mx            749
rocketmail.com         664
mail.com               559
earthlink.net          514
gmail                  496
outlook.es             438
mac.com                436
juno.com               322
aim.com                315
windstream.net         30

LEN de id_27: 3
Suma de NA: 0
U           585371
Found         5155
NotFound        14
Name: id_27, dtype: int64 

LEN de id_28: 3
Suma de NA: 0
U        449562
Found     76232
New       64746
Name: id_28, dtype: int64 

LEN de id_29: 3
Suma de NA: 0
U           449562
Found        74926
NotFound     66052
Name: id_29, dtype: int64 

LEN de id_30: 76
Suma de NA: 0
U                   512975
Windows 10           21155
Windows 7            13110
iOS 11.2.1            3722
iOS 11.1.2            3699
                     ...  
func                    10
iOS 11.4.0               5
Mac OS X 10_13_5         4
Windows                  3
iOS 11.4.1               1
Name: id_30, Length: 76, dtype: int64 

LEN de id_31: 131
Suma de NA: 0
U                        450258
chrome 63.0               22000
mobile safari 11.0        13423
mobile safari generic     11474
ie 11.0 for desktop        9030
                          ...  
cyberfox                      1
LG/K-200                      1
Cherry  

In [250]:
train_imputed['P_emaildomain_0'] = train_imputed['P_emaildomain'].apply(lambda x: x.split('.')[0])
train_imputed['P_emaildomain_1'] = train_imputed['P_emaildomain'].apply(lambda x: '.'.join(x.split('.')[1:]))
train_imputed['R_emaildomain_0'] = train_imputed['R_emaildomain'].apply(lambda x: x.split('.')[0])
train_imputed['R_emaildomain_1'] = train_imputed['R_emaildomain'].apply(lambda x: '.'.join(x.split('.')[1:]))

In [226]:
def proc_id_30_0(x):
    x = x.lower()
    try:
        if 'windows' in x or 'android' in x:
            return x.split()[1]
        elif 'ios' in x:
            return x.split()[1].split('.')[0]
        elif 'mac' in x:
            return x.split()[-1]
        elif 'linux' in x:
            return 'linux'
        else:
            return 'other'
    except:
        return 'other'
    
def proc_id_30_1(x):
    x = x.lower()
    if 'windows' in x:
        return 'windows'
    elif 'ios' in x:
        return 'ios'
    elif 'android' in x:
        return 'android'
    elif 'mac' in x:
        return 'mac'
    elif 'linux' in x:
        return 'linux'
    else:
        return 'other'

In [227]:
train_imputed['proc_id_30_0'] = train_imputed['id_30'].apply(lambda x: proc_id_30_0(x))
train_imputed['proc_id_30_1'] = train_imputed['id_30'].apply(lambda x: proc_id_30_1(x))

In [229]:
def proc_id_31_0(x):
    x = x.lower()
    if 'chrome' in x and not 'android' in x:
        return 'chrome'
    elif 'mobile' in x and 'safari' in x:
        return 'safari_mobile'
    elif 'ie' in x:
        return 'ie'
    elif 'safari' in x and not 'mobile' in x:
        return 'safari'
    elif 'chrome' in x and 'android' in x:
        return 'chrome_android'
    elif 'edge' in x:
        return 'edge'
    elif 'firefox' in x:
        return 'firefox'
    elif 'samsung' in x:
        return 'samsung'
    elif 'other' in x:
        return 'other'
    elif 'opera' in x:
        return 'opera'
    elif 'android' in x and not 'chrome' in x:
        return 'android'
    elif 'chrome' in x and 'ios' in x:
        return 'chrome_ios'
    elif 'google' in x:
        return 'google'
    else:
        return 'other'

In [230]:
train_imputed['proc_id_31_0'] = train_imputed['id_31'].apply(lambda x: proc_id_31_0(x))

In [234]:
train_imputed['TransactionDT'].median()

7306527.5

In [251]:
new_cat_ft = list()
for i, j in zip(train_imputed.dtypes, train_imputed.columns):
    if i in ['O', 'object']:
        new_cat_ft.append(j)
        
new_cat_ft = list(set(new_cat_ft)-set(cat_ft))
new_cat_ft = cat_ft + new_cat_ft
len(new_cat_ft)

56

In [231]:
train_imputed.head(10)

,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo,proc_id_30_0,proc_id_30_1,proc_id_31_0
0,0,86400,68.5,W,13926,-1.0,150.0,discover,142.0,credit,...,match_status:-2,U,U,U,U,U,U,other,other,other
1,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,credit,...,match_status:-2,U,U,U,U,U,U,other,other,other
2,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,debit,...,match_status:-2,U,U,U,U,U,U,other,other,other
3,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,debit,...,match_status:-2,U,U,U,U,U,U,other,other,other
4,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,credit,...,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M,7.0,android,samsung
5,0,86510,49.0,W,5937,555.0,150.0,visa,226.0,debit,...,match_status:-2,U,U,U,U,U,U,other,other,other
6,0,86522,159.0,W,12308,360.0,150.0,visa,166.0,debit,...,match_status:-2,U,U,U,U,U,U,other,other,other
7,0,86529,422.5,W,12695,490.0,150.0,visa,226.0,debit,...,match_status:-2,U,U,U,U,U,U,other,other,other
8,0,86535,15.0,H,2803,100.0,150.0,visa,226.0,debit,...,match_status:1,T,F,F,T,mobile,iOS Device,11,ios,safari_mobile
9,0,86536,117.0,W,17399,111.0,150.0,mastercard,224.0,debit,...,match_status:-2,U,U,U,U,U,U,other,other,other


In [309]:
for ft in new_cat_ft:
    train_imputed2 = freq_encoder(train_imputed2, ft, str(ft+'_fe'), min_freq = 0.0005)
    print('{0}: {1}'.format(ft, len(train_imputed2[ft+'_fe'].value_counts())))

ProductCD: 5
P_emaildomain: 34
R_emaildomain: 21
card1: 310
card2: 167
card3: 13
card4: 5
card5: 29
card6: 4
addr1: 60
addr2: 5
M1: 3
M2: 3
M3: 3
M4: 4
M5: 3
M6: 3
M7: 3
M8: 3
M9: 3
DeviceType: 3
DeviceInfo: 8
id_12: 3
id_13: 21
id_14: 8
id_15: 4
id_16: 3
id_17: 7
id_18: 7
id_19: 43
id_20: 57
id_21: 3
id_22: 4
id_23: 4
id_24: 4
id_25: 4
id_26: 9
id_27: 3
id_28: 3
id_29: 3
id_30: 33
id_31: 40
id_32: 4
id_33: 26
id_34: 5
id_35: 3
id_36: 3
id_37: 3
id_38: 3
proc_id_31_0: 11
P_emaildomain_1: 10
R_emaildomain_0: 18
R_emaildomain_1: 8
P_emaildomain_0: 30
proc_id_30_0: 22
proc_id_30_1: 6


In [305]:
train_imputed2.head()

,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,id_36_fe,id_37_fe,id_38_fe,proc_id_31_0_fe,P_emaildomain_1_fe,R_emaildomain_0_fe,R_emaildomain_1_fe,P_emaildomain_0_fe,proc_id_30_0_fe,proc_id_30_1_fe
0,0,86400,68.5,W,13926,-1.0,150.0,discover,142.0,credit,...,0,0,0,0,1,0,0,2,0,0
1,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,credit,...,0,0,0,0,0,0,0,0,0,0
2,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,debit,...,0,0,0,0,0,0,0,8,0,0
3,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,debit,...,0,0,0,0,0,0,0,1,0,0
4,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,credit,...,1,1,2,8,0,0,0,0,4,4


In [269]:
num_ft_imput_dict = dict()
for c in train_imputed.columns:
    if c not in new_cat_ft and train_imputed[c].isna().sum() > 0:
        mdn = train_imputed[c].median()
        num_ft_imput_dict[c] = mdn

In [271]:
train_imputed2 = train_imputed.fillna(num_ft_imput_dict)

In [310]:
train_imputed2.head()

,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,id_36_fe,id_37_fe,id_38_fe,proc_id_31_0_fe,P_emaildomain_1_fe,R_emaildomain_0_fe,R_emaildomain_1_fe,P_emaildomain_0_fe,proc_id_30_0_fe,proc_id_30_1_fe
0,0,86400,68.5,W,13926,-1.0,150.0,discover,142.0,credit,...,0,0,0,0,1,0,0,2,0,0
1,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,credit,...,0,0,0,0,0,0,0,0,0,0
2,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,debit,...,0,0,0,0,0,0,0,8,0,0
3,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,debit,...,0,0,0,0,0,0,0,1,0,0
4,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,credit,...,1,1,2,8,0,0,0,0,4,4


In [277]:
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import HashingVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve, auc
from sklearn.preprocessing import MinMaxScaler
from scipy.sparse import hstack, csr_matrix, vstack
import lightgbm as lgb

In [353]:
params = {
    'max_depth': 7,
    'metric': ['PR_AUC', 'auc'],
    'first_metric_only': True,
    'n_estimators': 20000,
    'learning_rate': 0.05,
    'colsample_bytree': 0.8,
    'objective': 'xentropy',
    'n_jobs': -1,
    'seed': 42,
    'bagging_fraction': 0.8,
    'lambda_l1': 0,
    'lambda_l2': 0,
}

In [354]:
def custom_loss(y_pred, y_true):
    precision, recall, thresholds = precision_recall_curve(np.where(y_true >= 0.5, 1, 0), y_pred)
    AUC = auc(recall, precision)
    if AUC != AUC:
        AUC = 0
    return 'PR_AUC', AUC, True

In [355]:
lgb_model = lgb.LGBMClassifier(**params)

In [356]:
X = train_imputed2[X_cols]
y = train_imputed2.isFraud

In [357]:
X_cols = [x for x in train_imputed2.columns if x not in new_cat_ft+['isFraud', 'TransactionDT', 'TransactionID']]
X_train, X_test, y_train, y_test = train_test_split(train_imputed2[X_cols], train_imputed2.isFraud,
                                                        test_size=0.2,
                                                        random_state=42)

In [358]:
# X.shape[0]*0.8

In [359]:
# X_train = X.iloc[:int(X.shape[0]*0.8),:]
# X_test = X.iloc[int(X.shape[0]*0.8):,:]
# y_train = y[:int(X.shape[0]*0.8)]
# y_test = y[int(X.shape[0]*0.8):]

In [360]:
lgb_model.fit(X_train,
                   y_train,
                   eval_set=[(X_test, y_test)],
                   verbose=50,
                   early_stopping_rounds=50,
                   eval_metric=custom_loss
                  )

Training until validation scores don't improve for 50 rounds.
[50]	valid_0's auc: 0.896637	valid_0's PR_AUC: 0.605812
[100]	valid_0's auc: 0.912792	valid_0's PR_AUC: 0.634008
[150]	valid_0's auc: 0.919693	valid_0's PR_AUC: 0.649791
[200]	valid_0's auc: 0.925025	valid_0's PR_AUC: 0.664403
[250]	valid_0's auc: 0.929158	valid_0's PR_AUC: 0.672901
[300]	valid_0's auc: 0.932273	valid_0's PR_AUC: 0.68292
[350]	valid_0's auc: 0.934585	valid_0's PR_AUC: 0.690099
[400]	valid_0's auc: 0.937438	valid_0's PR_AUC: 0.696055
[450]	valid_0's auc: 0.939681	valid_0's PR_AUC: 0.702555
[500]	valid_0's auc: 0.941324	valid_0's PR_AUC: 0.706891
[550]	valid_0's auc: 0.943532	valid_0's PR_AUC: 0.711418
[600]	valid_0's auc: 0.944757	valid_0's PR_AUC: 0.715155
[650]	valid_0's auc: 0.94632	valid_0's PR_AUC: 0.718552
[700]	valid_0's auc: 0.947891	valid_0's PR_AUC: 0.722093
[750]	valid_0's auc: 0.948816	valid_0's PR_AUC: 0.724547
[800]	valid_0's auc: 0.950269	valid_0's PR_AUC: 0.728019
[850]	valid_0's auc: 0.951407

LGBMClassifier(bagging_fraction=0.7, boosting_type='gbdt', class_weight=None,
               colsample_bytree=0.7, first_metric_only=True,
               importance_type='split', lambda_l1=0, lambda_l2=0,
               learning_rate=0.05, max_depth=7, metric=['PR_AUC', 'auc'],
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=20000, n_jobs=-1, num_leaves=31,
               objective='xentropy', random_state=None, reg_alpha=0.0,
               reg_lambda=0.0, seed=42, silent=True, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)

In [351]:
del X_train
del X_test
del y_train
del y_test

In [352]:
gc.collect()

20

In [254]:
import datetime

In [264]:
START_DATE = '2017-12-01'
startdate = datetime.datetime.strptime(START_DATE, '%Y-%m-%d')
X_train = pd.DataFrame()
X_train['TransactionDT'] = train_imputed['TransactionDT'].apply(lambda x: (startdate + datetime.timedelta(seconds = x)))

X_train['year'] = X_train['TransactionDT'].dt.year
X_train['month'] = X_train['TransactionDT'].dt.month
X_train['dow'] = X_train['TransactionDT'].dt.dayofweek
X_train['hour'] = X_train['TransactionDT'].dt.hour
X_train['day'] = X_train['TransactionDT'].dt.day

In [265]:
X_train.head(30)

,TransactionDT,year,month,dow,hour,day
0,2017-12-02 00:00:00,2017,12,5,0,2
1,2017-12-02 00:00:01,2017,12,5,0,2
2,2017-12-02 00:01:09,2017,12,5,0,2
3,2017-12-02 00:01:39,2017,12,5,0,2
4,2017-12-02 00:01:46,2017,12,5,0,2
5,2017-12-02 00:01:50,2017,12,5,0,2
6,2017-12-02 00:02:02,2017,12,5,0,2
7,2017-12-02 00:02:09,2017,12,5,0,2
8,2017-12-02 00:02:15,2017,12,5,0,2
9,2017-12-02 00:02:16,2017,12,5,0,2
